In [1]:
import numpy as np
import glob
import pickle


In [17]:
path_to_save_mri_data = '/usr/local/micapollo01/MIC/DATA/STUDENTS/mvhave7/Results/Preprocessing/mri/'
path_to_save_grappa_data = '/usr/local/micapollo01/MIC/DATA/STUDENTS/mvhave7/Results/Preprocessing/grappa/'


file_paths_train = sorted(glob.glob(path_to_save_mri_data+"training_data_GrappaNet_16_coils_batch_*.npy"))[50]
file_paths_train_GT = sorted(glob.glob(path_to_save_mri_data+"training_data_GT_GrappaNet_16_colis_batch_*.npy"))[50]
file_paths_val = sorted(glob.glob(path_to_save_mri_data+"validation_data_GrappaNet_16_coils_batch_*.npy"))[50]
file_paths_val_GT = sorted(glob.glob(path_to_save_mri_data+"validation_data_GT_GrappaNet_16_colis_batch_*.npy"))[50]
file_paths_grappa_indx_train = sorted(glob.glob(path_to_save_grappa_data+"grappa_train_indx_GrappaNet_16_coils_batch_*.npy"))[50]
file_paths_grappa_indx_val = sorted(glob.glob(path_to_save_grappa_data+"grappa_test_indx_GrappaNet_16_colis_batch_*.npy"))[50]
file_paths_grappa_wt = sorted(glob.glob(path_to_save_grappa_data+"grappa_wt_batch_*.pickle"))[50]
file_paths_grappa_p = sorted(glob.glob(path_to_save_grappa_data+"grappa_p_batch_*.pickle"))[50]


In [18]:
x_train = np.load(file_paths_train)
y_train = np.load(file_paths_train_GT)
grappa_train_indx = np.load(file_paths_grappa_indx_train)
with open(file_paths_grappa_wt, 'rb') as handle:
    grappa_wt = pickle.load(handle)
with open(file_paths_grappa_p, 'rb') as handle:
    grappa_p = pickle.load(handle)
x_test = np.load(file_paths_val)
y_test = np.load(file_paths_val_GT)
grappa_test_indx = np.load(file_paths_grappa_indx_val)
with open(file_paths_grappa_wt, 'rb') as handle:
    grappa_wt = pickle.load(handle)
with open(file_paths_grappa_p, 'rb') as handle:
    grappa_p = pickle.load(handle)


In [19]:
print(x_train.dtype)
print(y_train.dtype)
print(grappa_train_indx.dtype)
print(grappa_test_indx.dtype)
print(x_test.dtype)
print(y_test.dtype)


float32
float32
int64
int64
float32
float32


In [20]:
print(x_train.shape)
print(y_train.shape)
print(grappa_train_indx.shape)
print(grappa_test_indx.shape)
print(x_test.shape)
print(y_test.shape)


(138, 640, 320, 32)
(138, 640, 320)
(138,)
(16,)
(16, 640, 320, 32)
(16, 640, 320)


In [21]:
print(type(x_train))
print(type(y_train))
print(type(grappa_train_indx))
print(type(grappa_test_indx))
print(type(x_test))
print(type(y_test))
print(type(grappa_wt))
print(type(grappa_p))


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'list'>
<class 'list'>
